In [14]:
# Standard library imports
import os                  # Provides functions for interacting with the operating system (e.g., environment variables, file paths)
import json                # Handles JSON data operations (serialization and deserialization)
import requests            # Library for making HTTP requests (e.g., calling APIs)
import getpass             # Provides secure password input functionality without disclosing them

# Data processing
import pandas as pd                       # pandas library for data analysis and manipulation

# Data validation
from pydantic import BaseModel, Field     # Tools for data validation and settings management

# Jupyter notebook display utilities
from IPython.display import Image, display                                   # Tools for displaying images in notebooks

# Date and time handling
from datetime import datetime, timedelta  # Classes for working with dates and times (e.g., handling dates and time calculations)

# Type hinting imports
from typing import Dict, List, Any, Optional, Tuple  # rovides type hints for better code readability and static analysis

# LangChain modules for working with LLMs and embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings                   # Provides OpenAI-based chat and embedding models
from langchain.schema import Document                                       # Defines the structure of a Document schema for LangChain
from langchain.text_splitter import RecursiveCharacterTextSplitter          # Text splitting utility - splits large text into smaller chunks for processing
from langchain.vectorstores import FAISS                                    # Vector store - FAISS (Facebook AI Similarity Search) for vector-based retrieval

# LangChain chain-related imports
from langchain.chains import create_retrieval_chain                         # Creates chains for document retrieval to eventually create a retrieval-based LLM chain
from langchain.chains.combine_documents import create_stuff_documents_chain # Document combination utility (combines retrieved documents for processing)

# Prompt handling imports
from langchain.prompts import ChatPromptTemplate, PromptTemplate  # Defines prompt templates for LLM interactions

# Graph-related imports
from langgraph.graph import END, StateGraph                                  # Graph components for workflow
 
# LangChain core components
from langchain_core.messages import HumanMessage, AIMessage                  # Message types for chat (messages exchanged between human and AI)
from langchain_core.output_parsers import StrOutputParser                    # Parser for string outputs from LLM responses 
from langchain_core.runnables import RunnablePassthrough                     # Utility for chain composition allowing simple passthrough execution of functions
from langchain_core.messages import BaseMessage

# Langgraph Graph imports
from langgraph.graph import END, StateGraph  # Constructs a state graph for managing execution flow in a pipeline

# Amadeus API client for fetching travel-related data
from amadeus import Client, ResponseError                       # Client interacts with the Amadeus API, ResponseError handles exceptions

In [ ]:
# API Keys and Configuration

# OpenAI API credentials
# Prompt user to enter their OPENAI API Key
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter the OpenAI API Key:")

# Amadeus API credentials

# Prompt user to enter their Amadeus Client ID (API Key)
os.environ["AMADEUS_CLIENT_ID"] = getpass.getpass("Enter the Amadeus Client ID/API Key:")
# Prompt user to enter their Amadeus Client Secret (API Secret)
os.environ["AMADEUS_CLIENT_SECRET"] = getpass.getpass("Enter the Amadeus Client Secret/API Secret:")

# Initialize Amadeus client
amadeus = Client(
    client_id=os.environ["AMADEUS_CLIENT_ID"],
    client_secret=os.environ["AMADEUS_CLIENT_SECRET"]
)

# amadeus = Client(
#     client_id=AMADEUS_CLIENT_ID,
#     client_secret=AMADEUS_CLIENT_SECRET
# )

In [16]:
## Step 1: Building the Knowledge Base with Amadeus API Data
def get_amadeus_access_token():
    """Get Amadeus API access token."""
    auth_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_CLIENT_ID,
        "client_secret": AMADEUS_CLIENT_SECRET
    }
    
    response = requests.post(auth_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        print(f"Failed to get access token: {response.json()}")
        return None

In [17]:
### Fetch Popular Destinations Data
def fetch_popular_destinations(city_code):
    """Fetch popular travel destinations using Amadeus API."""
    try:
        response = amadeus.shopping.flight_destinations.get(         
            origin=city_code
        )
        destinations = response.data
        
        # Transform into documents for our knowledge base
        documents = []
        for dest in destinations:
            content = f"""
            Destination: {dest['destination']}
            Departure Date: {dest.get('departureDate', 'N/A')}
            Return Date: {dest.get('returnDate', 'N/A')}
            Price: {dest.get('price', {}).get('total', 'N/A')} {dest.get('price', {}).get('currency', 'USD')}
            """
            doc = Document(
                page_content=content,
                metadata={
                    "type": "popular_destination",
                    "destination_code": dest['destination'],
                    "price": dest.get('price', {}).get('total', 'N/A')
                }
            )
            documents.append(doc)
        return documents
    except ResponseError as error:
        print(f"Error fetching popular destinations: {error}")
        return []

In [ ]:
# # Checking output of this function
# tmp_list = []
# tmp_list = fetch_popular_destinations('NYC')
# len(tmp_list)
# tmp_list

In [18]:
### Fetch Hotel Information
# Rating: {hotel_info.get('rating', 'N/A')}
# Price: {price_info.get('total', 'N/A')} {price_info.get('currency', 'USD')}
# Description: {hotel_info.get('description', {}).get('text', 'No description available')}
# Amenities: {', '.join(hotel_info.get('amenities', ['N/A']))}

def fetch_hotel_information(city_code):
    """Fetch hotel information for a specific city."""
    try:
        response = amadeus.reference_data.locations.hotels.by_city.get(cityCode=city_code)
        # print(len(response.data))
        # print(response.data)

        hotels = response.data
        
        # Transform into documents
        documents = []
        for hotel in hotels:
                     
            content = f"""
            Hotel: {hotel.get('name', 'N/A')}
            Location: {city_code}
            Chain: {hotel.get('chainCode', 'N/A')}
            Hotel ID: {hotel.get('hotelId', 'N/A')}
            Coordinates: {hotel.get('geoCode', {}).get('latitude', 'N/A')}, {hotel.get('geoCode', {}).get('longitude', 'N/A')}
            Country: {hotel.get('address', {}).get('countryCode', 'N/A')}
            Last Updated: {hotel.get('lastUpdate', 'N/A')}
            """
            
            doc = Document(
                page_content=content,
                metadata={
                    "type": "hotel_information",
                    "hotel_id": hotel.get('hotelId', 'unknown'),
                    "city_code": city_code,
                    "chain_code": hotel.get('chainCode', 'N/A')
                    # "rating": hotel_info.get('rating', 'N/A')
                }
            )
            documents.append(doc)
        return documents
    except ResponseError as error:
        print(f"Error fetching hotel information: {error}")
        return []

In [19]:
### Fetch Flight Offers
def fetch_flight_offers(origin, destination, departure_date):
    """Fetch flight offers between two destinations."""
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate=departure_date,
            adults=1,
            max=10
        )
        flights = response.data
        
        # Transform into documents
        documents = []
        for flight in flights:
            # Extract key information
            itineraries = flight.get('itineraries', [])
            price_info = flight.get('price', {})
            
            segments_info = []
            for itinerary in itineraries:
                for segment in itinerary.get('segments', []):
                    departure = segment.get('departure', {})
                    arrival = segment.get('arrival', {})
                    carrier = segment.get('carrierCode', 'Unknown')
                    
                    segment_info = f"""
                    Flight: {carrier} {segment.get('number', 'N/A')}
                    From: {departure.get('iataCode', 'N/A')} at {departure.get('at', 'N/A')}
                    To: {arrival.get('iataCode', 'N/A')} at {arrival.get('at', 'N/A')}
                    """
                    segments_info.append(segment_info)
            
            content = f"""
            Route: {origin} to {destination}
            Departure Date: {departure_date}
            Price: {price_info.get('total', 'N/A')} {price_info.get('currency', 'EUR')}
            Flight Details:
            {"".join(segments_info)}
            """
            
            doc = Document(
                page_content=content,
                metadata={
                    "type": "flight_offer",
                    "origin": origin,
                    "destination": destination,
                    "departure_date": departure_date,
                    "price": price_info.get('total', 'N/A')
                }
            )
            documents.append(doc)
        return documents
    except ResponseError as error:
        print(f"Error fetching flight offers: {error}")
        return []

In [20]:
### Fetch Travel Advisories
def fetch_travel_advisories(country_code):
    """Fetch travel advisories for a specific country."""
    try:
        response = amadeus.safety.safety_rated_locations.get(
            safetyRatedLocationType='COUNTRY',
            countryCode=country_code
        )
        advisories = response.data
        
        # Use the correct API endpoint for safety information
   
        response = amadeus.reference_data.locations.safety_rated_locations.get(
            latitude=latitude,
            longitude=longitude
        )

        advisories = response.data        
        
        documents = []
        for advisory in advisories:
            safety_scores = advisory.get('safetyScores', {})
            
            content = f"""
            Country: {country_code}
            Overall Safety Score: {safety_scores.get('overall', 'N/A')}
            Physical Harm Risk: {safety_scores.get('physicalHarm', 'N/A')}
            Theft Risk: {safety_scores.get('theft', 'N/A')}
            Political Unrest Risk: {safety_scores.get('politicalFreedom', 'N/A')}
            Health Risk: {safety_scores.get('health', 'N/A')}
            Last Updated: {advisory.get('updatedDateTime', 'N/A')}
            """
            
            doc = Document(
                page_content=content,
                metadata={
                    "type": "travel_advisory",
                    "country_code": country_code,
                    "overall_safety": safety_scores.get('overall', 'N/A')
                }
            )
            documents.append(doc)
        return documents
        
    except ResponseError as error:
        print(f"Error fetching travel advisories: {error}")
        return []

In [21]:
### Collecting and Preparing Static Travel Information
# Let's also include static travel information for our knowledge base:
def prepare_destination_info():
    """Prepare static destination information."""
    # Prepare a static list of destinations with some fields of interest for travel planning
    destinations = [
        {
            "city": "Paris",
            "country": "France",
            "description": "Known as the City of Light, Paris is famous for the Eiffel Tower, Louvre Museum, and exquisite cuisine. Best time to visit is April-June or September-October for mild weather and fewer crowds.",
            "attractions": ["Eiffel Tower", "Louvre Museum", "Notre-Dame Cathedral", "Montmartre", "Champs-Élysées"],
            "cuisine": ["Croissants", "Escargot", "Coq au Vin", "Macarons", "French Wine"],
            "transportation": "Excellent public transportation with Metro, buses and RER trains. The Paris Visite travel pass offers unlimited travel on all transport networks.",
            "weather": "Temperate climate with mild winters and warm summers. Spring (March-May) and Fall (September-November) are particularly pleasant."
        },
        {
            "city": "Tokyo",
            "country": "Japan",
            "description": "Tokyo is a fascinating blend of ultramodern and traditional, with neon-lit skyscrapers coexisting with historic temples. Best time to visit is March-April for cherry blossoms or October-November for autumn colors.",
            "attractions": ["Tokyo Skytree", "Senso-ji Temple", "Meiji Shrine", "Shibuya Crossing", "Imperial Palace"],
            "cuisine": ["Sushi", "Ramen", "Tempura", "Yakitori", "Matcha desserts"],
            "transportation": "Highly efficient train and subway system. The Japan Rail Pass can be cost-effective for travelers. Taxis are clean but expensive.",
            "weather": "Four distinct seasons, with hot humid summers and cold winters. Spring and autumn are the most comfortable seasons."
        },
        # Add more destinations as needed
    ] 
    
    # Prepare an empty list where Document objects corresponding to each city in the destinations list will be stored
    documents = []
    for dest in destinations:
        content = f"""
        Destination: {dest['city']}, {dest['country']}
        
        Description: {dest['description']}
        
        Top Attractions: {', '.join(dest['attractions'])}
        
        Local Cuisine: {', '.join(dest['cuisine'])}
        
        Transportation: {dest['transportation']}
        
        Weather: {dest['weather']}
        """
        
        # The Document object is imported from langchain.schema package
        # Document is a class for storing a piece of text and associated metadata.
        # The Document object being created below will have 2 core components - 
        # page_content: a formatted string containing destination (city and country), description (of the city), 
        #               top attractions (comma-separated), local cuisine (comma-separated), transportation, and weather
        # metadata: a dictionary containing type, city, country
        doc = Document(
            page_content=content,
            metadata={
                "type": "destination_info", # category identifier used to filter documents by type
                "city": dest['city'],
                "country": dest['country']
            }
        )
        documents.append(doc)
    
    return documents

In [48]:
### Building the Vector Store
# Now, let's build our vector store by combining all this travel information:
def build_travel_knowledge_base():
    """Build the complete travel knowledge base."""
    all_documents = []
    
    # Collect static destination information
    # We are using the extend function of a list object to extract every single element (document) in the returned list of Document objects
    # and add them individually to the all_documents list.
    # In contrast to append(), which adds the entire iterable as a single element, extend() adds each element individually. 
    all_documents.extend(prepare_destination_info()) 
    
    # Collect dynamic information from Amadeus API
    # Popular destinations from NYC
    # We are adding elements of popular destinations from NYC to the base
    all_documents.extend(fetch_popular_destinations('NYC'))
    
    # # Sample hotel information for major cities
    for city_code in ['PAR', 'LON', 'NYC', 'TYO', 'ROM']:
        all_documents.extend(fetch_hotel_information(city_code))
    
    # Sample flight offers for popular routes
    next_month = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
    
    popular_routes = [
        ('NYC', 'LON'), ('NYC', 'PAR'), ('NYC', 'ROM'),
        ('LON', 'PAR'), ('LON', 'ROM'), ('PAR', 'ROM')
    ]
    
    for origin, destination in popular_routes:
        all_documents.extend(fetch_flight_offers(origin, destination, next_month))
    
    # # # Sample travel advisories
    # for country_code in ['FR', 'GB', 'US', 'JP', 'IT']:
    #     all_documents.extend(fetch_travel_advisories(country_code))
    
    # Split documents for better embedding
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    split_documents = text_splitter.split_documents(all_documents)
    
    # Create vector store
    vector_store = FAISS.from_documents(
        documents=split_documents,
        embedding=OpenAIEmbeddings()
    )
    
    return vector_store